<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/YT_video_comment_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import random
import time

from bs4 import BeautifulSoup
import requests
import json
import re

import pandas as pd
import numpy as np

import operator
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.discovery import build
import datetime

In [ ]:
# Function to convert string values containing suffixes 'K', 'M', or 'B' to integers and extract numeric values.
def convert_to_int(value):
  """
    - If the value is NaN or an empty string, return 0.
    - If the value is a string:
      - Extract numeric digits from the string.
      - Convert the extracted digits to an integer.
      - If the string contains 'K', multiply the number by 1,000.
      - If the string contains 'M', multiply the number by 1,000,000.
      - If the string contains 'B', multiply the number by 1,000,000,000.
    - Return the converted integer value.

  """
  if pd.isna(value) or value == '':
      return 0
  if isinstance(value, str):
      # Extract numbers and convert them
      num = re.findall(r'\d+', value)
      if not num:
          return 0
      num = ''.join(num)
      if 'K' in value:
          return int(float(num) * 1000)
      if 'M' in value:
          return int(float(num) * 1000000)
      if 'B' in value:
          return int(float(num) * 1000000000)
      return int(num)
  return int(value)

In [ ]:
# Initialize YouTube API client
def initialize_youtube_api(api_key):
    return build("youtube", "v3", developerKey=api_key)

def init_webdriver():
    """Initializes and returns a Chrome WebDriver instance with options."""
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chromedriver_autoinstaller.install()
        driver = webdriver.Chrome(options=chrome_options)

        print("WebDriver initialized successfully")  # Confirm initialization
        return driver
    except Exception as e:
        print(f"Failed to initialize WebDriver: {e}")
        raise

def close_webdriver(driver):
    """Closes the provided WebDriver instance."""
    print("WebDriver successfully closed")
    driver.quit()


In [ ]:
# Function to extract video ID from a YouTube URL
def extract_video_id(video_url):
    """
    Extracts the video ID from a given YouTube URL.

    Args:
        video_url (str): The full YouTube URL.

    Returns:
        str: The extracted video ID.
    """
    # The video ID is the part after 'v=' in the URL
    return video_url.split('v=')[-1]


In [ ]:
# # Test initialize_youtube_api
# youtube = initialize_youtube_api(api_key)
# try:
#   # Make a simple API request to check if it works
#   request = youtube.channels().list(
#       part="snippet,contentDetails,statistics",
#       id="UC_x5XG1OV2P6uZZ5FSM9Ttw"  # Example channel ID
#   )
#   response = request.execute()
#   print("YouTube API initialized successfully")
# except Exception as e:
#   print(f"YouTube API initialization failed: {e}")

# # Test init_webdriver and close_webdriver
# try:
#   driver = init_webdriver()
#   driver.get("https://www.google.com")  # Test loading a page
#   print("WebDriver test successful: Loaded Google homepage")
#   close_webdriver(driver)
#   print("WebDriver closed successfully")
# except Exception as e:
#   print(f"WebDriver test failed: {e}")


In [ ]:
def get_video_info(video_id):
    """
    Retrieves and displays detailed information about a YouTube video using the YouTube Data API.

    This function fetches data such as the video's title, URL, view count, like count, comment count,
    published date, a snippet of the description, and the thumbnail URL. It also formats the published
    date for better readability and prints all the extracted information.

    Args:
        video_id (str): The unique ID of the YouTube video for which information is to be retrieved.

    Returns:
        None: This function prints the extracted information directly and does not return any value.
    """

    # Request video details using the YouTube API
    response = youtube.videos().list(
        part="snippet,statistics",  # Request snippet and statistics data
        id=video_id
    ).execute()

    # Extract required information from the response
    video = response["items"][0]  # Get the first (and likely only) video item
    title = video["snippet"]["title"]  # Extract the video title
    view_count = video["statistics"]["viewCount"]  # Extract view count
    like_count = video["statistics"]["likeCount"]  # Extract like count
    commentCount = video["statistics"]["commentCount"]  # Extract comment count
    date_posted = video["snippet"]["publishedAt"]  # Extract publish date
    description = video["snippet"]["description"]  # Extract video description

    # Extract the thumbnail URL (using 'high' quality thumbnail)
    thumbnail_url = video["snippet"]["thumbnails"]["high"]["url"]

    # Convert date string to datetime object for formatting
    date_str = date_posted
    date_object = datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")

    # Format the date object to a more readable format
    date_posted = date_object.strftime("%Y-%m-%d %H:%M:%S")

    # Split the description into paragraphs using newlines
    #paragraphs = description.split("\n")


    # Get the first paragraph from the description
    # first_paragraph = paragraphs[0]
    first_paragraph = description

    # Construct the full YouTube video URL
    extracted_video_url = f"https://www.youtube.com/watch?v={video_id}"

    # # Print the extracted information
    # print("Title:", title)
    # print("Video URL:", extracted_video_url)
    # print("View count:", view_count)
    # print("Like count:", like_count)
    # print("Number of Comments:", commentCount)
    # print("Date posted:", date_posted)
    # print("Description:", first_paragraph)
    # print("Thumbnail URL:", thumbnail_url)
    # print("\n\n")
    return title, extracted_video_url, view_count, like_count, commentCount, date_posted, first_paragraph, thumbnail_url



In [ ]:
# Function to get the channel ID from a URL that has `channel handles`
def get_channel_id_from_url_YTAPI(api_key, channel_url):
    """
    Args:
      api_key: Your YouTube Data API key.
      channel_url: The YouTube URL containing the channel handle.

    Returns:
        The channel ID if found, otherwise None.
    """
    try:
        youtube = build('youtube', 'v3', developerKey=api_key)
        username = channel_url.split('/')[-1]
        request = youtube.search().list(
            part='snippet',
            q=username,
            type='channel',
            maxResults=1
        )
        response = request.execute()

        if 'items' in response and len(response['items']) > 0:
            channel_id = response['items'][0]['snippet']['channelId']
            return channel_id
        else:
            print(f"Channel ID not found for URL: {channel_url}")
            return None

    except googleapiclient.errors.HttpError as e:
        print(f"HTTP Error occurred: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# list_of_channels_urls = [
#     "https://www.youtube.com/@BestEverFoodReviewShow",
#     "https://www.youtube.com/@TheRamseyShow",
#     "https://www.youtube.com/@MrBallen",
#     "https://www.youtube.com/@MattWalsh",
#     "https://www.youtube.com/@TheBehaviorPanel",
#     "https://www.youtube.com/@VICE",
#     "https://www.youtube.com/@CNN",
#     "https://www.youtube.com/@Jeep",
#     "https://www.youtube.com/@MrBeast"
#     ]

In [ ]:
# # Test `get_channel_id_from_url_YTAPI` using the `list_of_channels_urls`
# for channel_url in list_of_channels_urls:
#   channel_id = get_channel_id_from_url_YTAPI(api_key, channel_url)
#   print(f"Channel URL: {channel_url}, Channel ID: {channel_id}")


In [ ]:
def get_top_videos(channel_id, api_key):
    """
    Retrieves the top 10 most viewed videos for a given YouTube channel.

    Args:
        channel_id (str): The ID of the YouTube channel.
        api_key (str): A valid YouTube Data API key.

    Returns:
        list: A list containing the video IDs of the top 10 most viewed videos.
    """

    # Construct the YouTube Data API request URL.
    url = f"https://www.googleapis.com/youtube/v3/search?key={api_key}&channelId={channel_id}&part=snippet,id&order=viewCount&maxResults=10"


    # Send a GET request to the YouTube Data API.
    response = requests.get(url)

    # Parse the JSON response from the API.
    data = response.json()

    top_videos_IDs = []
    for item in data['items']:
        # Extract the video ID from each search result item, handling potential KeyError exceptions.
        try:
            top_videos_IDs.append(item['id']['videoId'])
        except KeyError:
            continue # Skip items that don't have a 'videoId' (e.g., channel results)

    return top_videos_IDs

In [ ]:
# channel_url_id_list = [
#     {
#         'Channel URL': 'https://www.youtube.com/@BestEverFoodReviewShow',
#         'Channel ID': 'UCcAd5Np7fO8SeejB1FVKcYw'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@TheRamseyShow',
#         'Channel ID': 'UCzpwkXk_GlfmWntZ9v4l3Tg'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@MrBallen',
#         'Channel ID': 'UCtPrkXdtCM5DACLufB9jbsA'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@MattWalsh',
#         'Channel ID': 'UCO01ytfzgXYy4glnPJm4PPQ'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@TheBehaviorPanel',
#         'Channel ID': 'UCx_8ri2rYergbu_06VNSPlw'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@VICE',
#         'Channel ID': 'UCn8zNIfYAQNdrFRrr8oibKw'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@CNN',
#         'Channel ID': 'UCupvZG-5ko_eiXAupbDfxWw'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@Jeep',
#         'Channel ID': 'UCMWLdSdAyDcCy_OVzONKm0w'
#         },
#     {
#         'Channel URL': 'https://www.youtube.com/@MrBeast',
#         'Channel ID': 'UCX6OQ3DkcsbYNE6H8uQQuVA'
#         }
#     ]

In [ ]:
# # Test get_top_videos function
# for channel in channel_url_id_list:
#     channel_id = channel['Channel ID']
#     top_videos = get_top_videos(channel_id, api_key)
#     print(f"Top 10 videos for channel {channel['Channel URL']}:")
#     channel['Channel Top Videos'] = top_videos
#     for video_id in top_videos:
#         print(f"  - https://www.youtube.com/watch?v={video_id}")
#     print("-" * 30)

In [ ]:
# channel_url_id_list

In [ ]:
def get_video_comments_count(video_id, youtube):
    """
    Retrieves the number of comments for a YouTube video.

    Args:
        video_id: The ID of the YouTube video.
        youtube: An instance of the YouTube API client.

    Returns:
        The number of comments for the video, or 0 if not available.
    """
    response = youtube.videos().list(  # Makes an API call to the YouTube Data API's 'videos.list' method.
        part='statistics',  # Requests only the 'statistics' part of the video data.
        id=video_id  # Specifies the video ID to retrieve data for.
    ).execute()  # Executes the API request and gets the response.

    return int(response['items'][0]['statistics'].get('commentCount', 0))  # Extracts the 'commentCount' from the response and returns it as an integer.
                                                                              # If 'commentCount' is not found, it returns 0.



In [ ]:
# # Select a channel and get its top videos
# channel = channel_url_id_list[0]  # Example: Selecting the first channel
# top_videos = channel['Channel Top Videos']

# # Test get_video_comments_count function for the first two videos
# for video_id in top_videos[:2]:
#     comment_count = get_video_comments_count(video_id, youtube)
#     print(f"Video ID: {video_id}, Comment Count: {comment_count}")


In [ ]:
def get_comments_html(video_url):
    """
    Fetches the HTML content of the comments section from a YouTube video.

    This function initializes a WebDriver instance to open the provided YouTube video URL,
    scrolls down to load the comments section, and retrieves the HTML content of the loaded
    comments section.

    Args:
        video_url (str): The URL of the YouTube video from which to fetch comments.

    Returns:
        str: The HTML content of the comments section.

    Raises:
        TimeoutException: If the comments section does not load within the specified time.
    """
    driver = init_webdriver()
    driver.get(video_url)

    # Wait until the comments section is loaded
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ytd-comments')))

    # Scroll to the comments section to load initial comments
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Set initial values for dynamic loading
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    scroll_pause_time = 2  # Time to wait between scrolls
    max_scrolls = 100  # Increase the max number of scrolls to ensure all comments are loaded
    scroll_count = 0

    while scroll_count < max_scrolls:
        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait for new comments to load dynamically
        time.sleep(scroll_pause_time)  # Simple wait to allow comments to load

        # Check the new scroll height and compare it with the last height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            # If the height hasn't changed, try one more scroll to ensure all comments are loaded
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                # If the height still hasn't changed, we've reached the end
                print("All comments have been loaded.")
                break

        last_height = new_height
        scroll_count += 1

    # Get the HTML of the comments section
    comments_html = driver.page_source

    # Close the driver
    driver.quit()

    return comments_html

In [ ]:
# # Get the top videos for a channel
# channel = channel_url_id_list[0]
# top_videos = channel['Channel Top Videos']

# # Test get_comments_html function for the first video
# video_url = f"https://www.youtube.com/watch?v={top_videos[0]}"
# comments_html = get_comments_html(video_url)


In [ ]:
def get_comment_thread_renderers(comments_html):
    """
    Parses the provided HTML content to extract YouTube comment threads and their counts.

    This function uses BeautifulSoup to parse the HTML content of a YouTube video's comments section.
    It finds and prints the number of comments and the number of comment thread renderers (`ytd-comment-thread-renderer`).
    It then returns a list of all the `ytd-comment-thread-renderer` elements found in the HTML.

    Args:
        comments_html (str): The HTML content of the comments section of a YouTube video.

    Returns:
        list: A list of `ytd-comment-thread-renderer` elements found in the HTML.
    """

    soup = BeautifulSoup(comments_html, 'html.parser')

    # Find the span element with the specified class
    comment_count_span = soup.find('span', class_='style-scope yt-formatted-string')

    # Extract the text content of the span element
    comment_count = comment_count_span.text.strip()

    # Print or use the comment count
    print("Comment Count:", comment_count)

    # Find all occurrences of the ytd-comment-thread-renderer element
    comment_thread_renderers = soup.find_all('ytd-comment-thread-renderer', class_='style-scope ytd-item-section-renderer')

    # Count the number of occurrences
    comment_thread_count = len(comment_thread_renderers)

    # Print or use the comment thread count
    print("Number of ytd-comment-thread-renderer elements:", comment_thread_count)

    return comment_thread_renderers

In [ ]:
# # Test get_comment_thread_renderers function
# comment_thread_renderers = get_comment_thread_renderers(comments_html)


In [ ]:
def get_comments(comment_thread_renderers):
    comments = list()
    comments_data = list()
    # Iterate through each comment thread renderer
    for comment_thread_renderer in comment_thread_renderers:

        # Extracting the comment text
        comment_text_element = comment_thread_renderer.find('yt-attributed-string', id='content-text')
        comment_text = comment_text_element.get_text(strip=True) if comment_text_element else None

        # Extracting the number of likes
        like_count_element = comment_thread_renderer.find('span', class_='style-scope ytd-comment-engagement-bar')
        like_count = like_count_element.get_text(strip=True) if like_count_element else None

        # Extracting the number of replies
        reply_count_element = comment_thread_renderer.find('ytd-button-renderer', id='more-replies')
        reply_count = reply_count_element.get_text(strip=True) if reply_count_element else None

        comments.append(comment_text)

        comments_data.append(
            {
            "comment_text": comment_text,
            "like_count": like_count,
            "reply_count": reply_count

            }
        )

    return comments, comments_data



In [ ]:
# # Test get_comments function
# comments, comments_data = get_comments(comment_thread_renderers)
# df = pd.DataFrame(comments_data)
# # Fill missing and empty values with 0
# df['like_count'].replace('', 0, inplace=True)
# df['reply_count'].replace('', 0, inplace=True)
# df.fillna({'like_count': 0, 'reply_count': 0}, inplace=True)

# # Convert columns to integers
# df['like_count'] = df['like_count'].apply(convert_to_int)
# df['reply_count'] = df['reply_count'].apply(convert_to_int)

# df.head()

In [ ]:
# # Test get_comments_html2
# start_time = time.time()
# comments_html = get_comments_html2(video_url)
# end_time = time.time()
# print("Comments HTML retrieved.")
# print(f"Time taken: {end_time - start_time} seconds")


In [ ]:
# # Example YouTube video URL
# video_url = "https://www.youtube.com/watch?v=rkyj1U0n2TA"


# # Extract video ID
# video_id = extract_video_id(video_url)

# # Initialize YouTube API client
# api_key = userdata.get('YouTubeAPI_key')  # Replace with your API key if not using Colab
# youtube = initialize_youtube_api(api_key)



# # Retrieve and display video comments
# def get_video_comments(video_url):
#     """
#     Retrieves comments from the provided YouTube video URL.

#     Args:
#         video_url (str): The URL of the YouTube video.

#     Returns:
#         list: A list of comments and their data.
#     """
#     comments_html = get_comments_html(video_url)  # Get HTML of comments section
#     comment_thread_renderers = get_comment_thread_renderers(comments_html)  # Extract comment renderers
#     _, comments_data = get_comments(comment_thread_renderers)  # Extract comment data

#     return comments_data

# # Display video info
# print("\nVideo Information:\n")
# title, extracted_video_url, view_count, like_count, commentCount, date_posted, first_paragraph, thumbnail_url = get_video_info(video_id)
# print(f"Title: {title}")
# print(f"Video URL: {extracted_video_url}")
# print(f"View count: {view_count}")
# print(f"Like count: {like_count}")
# print(f"Number of Comments: {commentCount}")
# print(f"Date posted: {date_posted}")
# print(f"Description: {first_paragraph}")
# print(f"Thumbnail URL: {thumbnail_url}")
# print("\n")

# # Retrieve and display video comments in JSON format
# comments_data = get_video_comments(video_url)

# # Convert comments to JSON and display
# comments_json = json.dumps(comments_data, indent=4)
# print("\nVideo Comments (JSON Format):\n")
# print(comments_json)